In [1]:
# Import statements:
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from statistics import mode, mean
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras_tuner import HyperModel, RandomSearch
from tensorflow.keras import layers
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential

# Task 1: Re-implement the network described by Qian and Sejnowski in 1988

## 1.1. Data Collection

In [2]:
# Function for reading dataset from file and storing it as 2 strings - one for amino acids and one for secondary structures, 
# with groups split by spacer '|':
def read_data(file_name):
    group_started = False 
    amino_acids = ""
    secondary_structures = ""
    missing = 0 # Counter for the missing values
    num_groups = 0 # Counter for the number of groups in the dataset

    # Read the file line by line:
    with open(file_name, 'r') as f:
        for l in f:
            line = l.strip()

            # Stop reading when reached the end of file:
            if line == "<end>":
                break

            # Start of a new group of amino acids:
            if line == "<>":
                if group_started:
                    # Add spacers at the end of the group:
                    amino_acids += '|'
                    secondary_structures += '|'
                num_groups += 1
                group_started = True
                continue

            # End of a group of amino acids:
            if line == "end":
                continue

            # Write amino acids and secondary structures when a group has been started:
            if group_started:
                if line:
                    chars = line.split(" ")
                    if len(chars) < 2:
                        missing += 1
                    amino_acids += chars[0]
                    secondary_structures += chars[1]

    return amino_acids, secondary_structures, missing, num_groups

# Store the training dataset as 2 strings - one for amino acids and one for secondary structures, with groups split by spacer '|':
train_amino_acids, train_secondary_structures, num_train_missing, num_train_groups = read_data("Q_and_s_data/protein-secondary-structure.train.txt")
print("Number of missing values in the train set:", num_train_missing)
print("Number of protein groups in the train set:", num_train_groups)

# Store the testing dataset as 2 strings - one for amino acids and one for secondary structures, with groups split by spacer '|':
test_amino_acids, test_secondary_structures, num_test_missing, num_test_groups = read_data("Q_and_s_data/protein-secondary-structure.test.txt")
print("\nNumber of missing values in the test set:", num_test_missing)
print("Number of protein groups in the test set:", num_test_groups)

Number of missing values in the train set: 0
Number of protein groups in the train set: 111

Number of missing values in the test set: 0
Number of protein groups in the test set: 1


In [3]:
# Print the amino acids of the training set and verify correct format of storing:
print(train_amino_acids)

GVGTVPMTDYGNDVEYYGQVTIGTPGKSFNLNFDTGSSNLWVGSVQCQASGCKGGRDKFNPSDGSTFKATGYDASIGYGDGSASGVLGYDTVQVGGIDVTGGPQIQLAQRLGGGGFPGDNDGLLGLGFDTLSITPQSSTNAFDQVSAQGKVIQPVFVVYLAASNISDGDFTMPGWIDNKYGGTLLNTNIDAGEGYWALNVTGATADSTYLGAIFQAILDTGTSLLILPDEAAVGNLVGFAGAQDAALGGFVIACTSAGFKSIPWSIYSAIFEIITALGNAEDDSGCTSGIGASSLGEAILGDQFLKQQYVVFDRDNGIRLAPVA|AQCEATIESNDAMQYDLKEMVVDKSCKQFTVHLKHVGKMAKSVMGHNWVLTKEADKEGVATDGMNAGLAQDYVKAGDTRVIAHTKVIGGGESDSVTFDVSKLTPGEAYAYFCSFPGHWAMMKGTLKLSN|SVDIQGNDQMQFNTNAITVDKSCKQFTVNLSHPGNLPKNVMGHNWVLSTAADMQGVVTDGMASGLDKDYLKPDDSRVIAHTKLIGSGEKDSVTFDVSKLKEGEQYMFFCTFPGHSALMKGTLTLK|ALWQFNGMIKCKIPSSEPLLDFNNYGCYCGLGGSGTPVDDLDRCCQTHDNCYKQAKKLDSCKVLVDNPYTNNYSYSCSNNEITCSSENNACEAFICNCDRNAAICFSKVPYNKEHKNLDKKNC|HWGYGKHDGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKPLSVSYDQATSLRILNDGHAFNVEFDDSEDKAVLKGGPLDGTYRLIQFHFHWGSLDGEGSQHTVDKKKYAAELHLVHWNTKYGDFGKAVQQPDGLAVLGIFLKVGSAKPGLQKVVDVLDSIKTKGKSADFTNFDPRGLLPESLDYWTYPGSLTTPPLLECVTWIVLKEPISVSSEQVLKFRKLNFDGEGEPEELMVDNWRPAQPLKNRQIKASF|GGGARSGDDVVAKYCNACHGTGLLNAPKVGDSAAWKTR

In [4]:
# Print the secondary of the training set and verify correct format of storing:
print(train_secondary_structures)

___________________ee______ee__eee______ee_________________________________________ee____________________eeeee____________eee_______________hhhhhhh___________eee_____eee____________________________ee_______________ee________ee________________________________________________________________________ee_hhhh_____eee___eee_____|___eeeeee_________eeeee____eeeeeeee_____hhhh____eeee___hhhhhhhhh_________________ee________eeeeeee_________eeee__________eeeeeee_|_eeee___________eee_____eeeeeee______________eeee____hhhhhhhhhh_hhhh_____________________eeeeee_________eeee__________eeeeee_|_hhhhhhhhhhh_____hhhh__________________hhhhhhhhhhhhhhhh___hhhhh___________eeee__eeee_____hhhhhhhhhhhhhhhhhh________________|____________________________ee_____________eeee_______eeee____eeee________eeee______eeeeeeeeee__________ee______eeeeeeeee_____hhhh______eeeeeeeeeee___hhhhhhh___________eee________________ee_ee__________eeeee____eeeehhhhhhh______________________________ee__|_______hhhhhh__hhhh____________hhhhhhh

In [5]:
# Print the amino acids of the training set for each protein separately:
train_groups = train_amino_acids.split("|")
for l in train_groups:
    print(l, "\n")

GVGTVPMTDYGNDVEYYGQVTIGTPGKSFNLNFDTGSSNLWVGSVQCQASGCKGGRDKFNPSDGSTFKATGYDASIGYGDGSASGVLGYDTVQVGGIDVTGGPQIQLAQRLGGGGFPGDNDGLLGLGFDTLSITPQSSTNAFDQVSAQGKVIQPVFVVYLAASNISDGDFTMPGWIDNKYGGTLLNTNIDAGEGYWALNVTGATADSTYLGAIFQAILDTGTSLLILPDEAAVGNLVGFAGAQDAALGGFVIACTSAGFKSIPWSIYSAIFEIITALGNAEDDSGCTSGIGASSLGEAILGDQFLKQQYVVFDRDNGIRLAPVA 

AQCEATIESNDAMQYDLKEMVVDKSCKQFTVHLKHVGKMAKSVMGHNWVLTKEADKEGVATDGMNAGLAQDYVKAGDTRVIAHTKVIGGGESDSVTFDVSKLTPGEAYAYFCSFPGHWAMMKGTLKLSN 

SVDIQGNDQMQFNTNAITVDKSCKQFTVNLSHPGNLPKNVMGHNWVLSTAADMQGVVTDGMASGLDKDYLKPDDSRVIAHTKLIGSGEKDSVTFDVSKLKEGEQYMFFCTFPGHSALMKGTLTLK 

ALWQFNGMIKCKIPSSEPLLDFNNYGCYCGLGGSGTPVDDLDRCCQTHDNCYKQAKKLDSCKVLVDNPYTNNYSYSCSNNEITCSSENNACEAFICNCDRNAAICFSKVPYNKEHKNLDKKNC 

HWGYGKHDGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKPLSVSYDQATSLRILNDGHAFNVEFDDSEDKAVLKGGPLDGTYRLIQFHFHWGSLDGEGSQHTVDKKKYAAELHLVHWNTKYGDFGKAVQQPDGLAVLGIFLKVGSAKPGLQKVVDVLDSIKTKGKSADFTNFDPRGLLPESLDYWTYPGSLTTPPLLECVTWIVLKEPISVSSEQVLKFRKLNFDGEGEPEELMVDNWRPAQPLKNRQIKASF 

GGGARSGDDVVAKYCNACHGTGLLNAPK

In [6]:
# Print the secondary structures of the training set for each protein separately:
train_groups_sec = train_secondary_structures.split("|")
for l in train_groups_sec:
    print(l, "\n")

___________________ee______ee__eee______ee_________________________________________ee____________________eeeee____________eee_______________hhhhhhh___________eee_____eee____________________________ee_______________ee________ee________________________________________________________________________ee_hhhh_____eee___eee_____ 

___eeeeee_________eeeee____eeeeeeee_____hhhh____eeee___hhhhhhhhh_________________ee________eeeeeee_________eeee__________eeeeeee_ 

_eeee___________eee_____eeeeeee______________eeee____hhhhhhhhhh_hhhh_____________________eeeeee_________eeee__________eeeeee_ 

_hhhhhhhhhhh_____hhhh__________________hhhhhhhhhhhhhhhh___hhhhh___________eeee__eeee_____hhhhhhhhhhhhhhhhhh________________ 

____________________________ee_____________eeee_______eeee____eeee________eeee______eeeeeeeeee__________ee______eeeeeeeee_____hhhh______eeeeeeeeeee___hhhhhhh___________eee________________ee_ee__________eeeee____eeeehhhhhhh______________________________ee__ 

_______hhhhhh__hhhh_________

## 1.2. Data Analysis

In [7]:
# Print the types of amino acids and secondary structures found in the given dataset:
print("Amino acid symbols:", set(train_amino_acids + test_amino_acids))
print("Secndary structure symbols:", set(train_secondary_structures + test_secondary_structures))

Amino acid symbols: {'H', 'F', 'G', 'Y', 'Q', 'K', 'W', 'V', '|', 'C', 'A', 'R', 'M', 'N', 'I', 'P', 'E', 'L', 'T', 'S', 'D'}
Secndary structure symbols: {'h', '|', '_', 'e'}


**Secondary structures:'h' = alpha-helix, 'e' = beta-sheet, '_' = coil. Symbols '|' represent spacers.**

In [8]:
# Print the number of times each secondary structure appeared in the train set:
secondary_structures_counter_train = Counter(train_secondary_structures)
print("Secondary structure counts (train set):")
for char, count in secondary_structures_counter_train.items():
    if char != "|":
        print(f"'{char}': {count}")

# Print the number of times each secondary structure appeared in the test set:
secondary_structures_counter_test = Counter(test_secondary_structures)
print("\nSecondary structure counts (test set):")
for char, count in secondary_structures_counter_test.items():
    if char != "|":
        print(f"'{char}': {count}")

Secondary structure counts (train set):
'_': 9868
'e': 3636
'h': 4601

Secondary structure counts (test set):
'_': 182
'e': 18
'h': 106


In [9]:
# Calculate the size of each protein group of the train set:
train_group_sizes = [len(group.strip()) for group in train_groups]

# Print the size of each group:
print("Size of each group in the train set:")
for i, length in enumerate(train_group_sizes, 1):
    print(f"Group {i}: {length}")
print()

# Print the mode and mean sizes of train set's protein groups:
print("Mode group size:", mode(train_group_sizes))
print("Mean group size:", round(mean(train_group_sizes)))

Size of each group in the train set:
Group 1: 324
Group 2: 129
Group 3: 125
Group 4: 123
Group 5: 256
Group 6: 83
Group 7: 111
Group 8: 108
Group 9: 46
Group 10: 71
Group 11: 118
Group 12: 103
Group 13: 136
Group 14: 240
Group 15: 44
Group 16: 207
Group 17: 141
Group 18: 146
Group 19: 54
Group 20: 147
Group 21: 29
Group 22: 174
Group 23: 70
Group 24: 67
Group 25: 35
Group 26: 149
Group 27: 141
Group 28: 145
Group 29: 85
Group 30: 216
Group 31: 239
Group 32: 21
Group 33: 30
Group 34: 329
Group 35: 130
Group 36: 164
Group 37: 129
Group 38: 153
Group 39: 153
Group 40: 26
Group 41: 124
Group 42: 111
Group 43: 36
Group 44: 107
Group 45: 293
Group 46: 124
Group 47: 65
Group 48: 225
Group 49: 56
Group 50: 247
Group 51: 194
Group 52: 318
Group 53: 323
Group 54: 85
Group 55: 256
Group 56: 127
Group 57: 293
Group 58: 141
Group 59: 146
Group 60: 106
Group 61: 11
Group 62: 131
Group 63: 95
Group 64: 87
Group 65: 75
Group 66: 80
Group 67: 152
Group 68: 57
Group 69: 153
Group 70: 220
Group 71: 222
G

**It is evident that different protein groups vary in size. This is expected, because in nature proteins can vary in size.**

## 1.3. Data Pre-Processing

### 1.3.1. Sequence Encoding (Local Encoding)

In [10]:
# Store each amino acid type (available in the training set) in a string that is sorted in alphabetical order:
amino_acid_chars = ''.join(c for c in set(train_amino_acids) if c != '|')
amino_acid_chars = ''.join(sorted(amino_acid_chars))

# Map each amino acid type (available in the training set) to a unique integer: 
acids_to_ints = {acid: id for id, acid in enumerate(amino_acid_chars)}
acids_to_ints['|'] = 20
print("Mapping of amino acid types to integers:")
print(acids_to_ints)


# Store each secondary structure type (available in the training set) in a string that is sorted in alphabetical order:
secondary_structure_chars = "he_"

# Map each secondary structure type (available in the training set) to a unique integer: 
structures_to_ints = {structure: id for id, structure in enumerate(secondary_structure_chars)}
print("\nMapping of secondary structure types to integers:")
print(structures_to_ints)

Mapping of amino acid types to integers:
{'A': 0, 'C': 1, 'D': 2, 'E': 3, 'F': 4, 'G': 5, 'H': 6, 'I': 7, 'K': 8, 'L': 9, 'M': 10, 'N': 11, 'P': 12, 'Q': 13, 'R': 14, 'S': 15, 'T': 16, 'V': 17, 'W': 18, 'Y': 19, '|': 20}

Mapping of secondary structure types to integers:
{'h': 0, 'e': 1, '_': 2}


In [11]:
# Function for performing one-hot-encoding of a single amino acid:
def one_hot_encoding(acid_or_structure, mapping_to_ints):
    encoded = np.zeros(len(mapping_to_ints))
    encoded[mapping_to_ints[acid_or_structure]] = 1
    return encoded

### 1.3.2. Sliding Window Approach

In [12]:
# Function for retrieving the X and y sets for training the neural network using the sliding window approach,
# as described in the paper, with sliding window size = 13:
def sliding_window(amino_acids, secondary_structures):
    central_index = 13 // 2
    X = []
    y = []

    # Loop through the data window by window:
    for i in range(central_index, len(amino_acids) - central_index):
        # Get the window of amino acids:
        window_acids = amino_acids[i - central_index:i + central_index + 1]
        
        # Skip the current window if it contains a spacer (to only include those windows that belong to a single protein):
        if '|' in window_acids:
            continue

        # Apply local encoding to the amino acids of the window and store the window in the array with input features:
        encoded_acids = [one_hot_encoding(acid, acids_to_ints) for acid in window_acids]
        X.append(np.concatenate(encoded_acids))

        # Apply local encoding to the central secondary structure of the window and store the structure in the array with target values:
        encoded_structure = one_hot_encoding(secondary_structures[i], structures_to_ints)
        y.append(encoded_structure)
        
    return np.array(X), np.array(y)

# Get the input features and target values for the train set using the sliding window approach:
X_train, y_train = sliding_window(train_amino_acids, train_secondary_structures)

# Get the input features and target values for the test set using the sliding window approach:
X_test, y_test = sliding_window(test_amino_acids, test_secondary_structures)

## 1.4. Model Set-up

In [13]:
# Function for initializing the Neural Network model and its layers: 
def setup_model(num_units, num_layers, input_shape):
    # Initialize the model:
    if num_layers == 2:
        model = Sequential([
            Dense(num_units, activation='relu', input_shape=(input_shape,)),
            Dense(3, activation='softmax')
        ])
    else:
        model = Sequential([
            Dense(num_units, activation='relu', input_shape=(input_shape,)),
            Dense(num_units / 2, activation='relu'),
            Dense(3, activation='softmax')
        ])

    # Set the model's optimizer, loss function and training performance metrics:
    model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

## 1.5. Hyperparameter tuning

In [14]:
# Function for converting predicted probabilities for each class into one-hot local encoding:
def predictions_one_hot_encoding(predictions):
    return np.eye(3)[np.argmax(predictions, axis=1)]

# Function for calculating the Q3 performance metric:
def calculate_q3_score(actual_structures, predicted_structures):
    correct_predictions = np.equal(np.argmax(actual_structures, axis=1), np.argmax(predicted_structures, axis=1))
    return np.sum(correct_predictions) / len(actual_structures)

# Function for calculating correlation coefficients for alpha-helix, beta-sheet and coil structures:
def calculate_correlation_coefficients(y_true, y_pred):
    
    # Identify True Positives (TP), True Negatives (TN), False Positives (FP), and False Negatives (FN):
    TP = np.zeros(3)
    FP = np.zeros(3)
    TN = np.zeros(3)
    FN = np.zeros(3)
    for i in range(3):  # because there are 3 classes
        TP[i] = np.sum((y_true[:, i] == 1) & (y_pred[:, i] == 1))
        TN[i] = np.sum((y_true[:, i] == 0) & (y_pred[:, i] == 0))
        FP[i] = np.sum((y_true[:, i] == 0) & (y_pred[:, i] == 1))
        FN[i] = np.sum((y_true[:, i] == 1) & (y_pred[:, i] == 0))


    # Calculate the correlation coefficients for each secondary structure class:
    correlation_coefficients = {}
    for i, structure in enumerate(['alpha-helix', 'beta-sheet', 'coil']):
        # denominator = np.sqrt((TN[i] + FN[i]) * (TN[i] + FP[i]) * (TP[i] + FN[i]) * (TP[i] + FP[i]))
        denominator = np.sqrt((TP[i] + FP[i]) * (TP[i] + FN[i]) * (TN[i] + FP[i]) * (TN[i] + FN[i]))
        # Handle cases when the denominator is equal to 0:
        if denominator == 0:
            correlation_coefficients[structure] = 0
        # Otherwise, apply the full formula:
        else:
            numerator = TP[i] * TN[i] - FP[i] * FN[i]
            correlation_coefficients[structure] = numerator / denominator

    return correlation_coefficients

In [104]:
# Define the hyperparameter configurations that will be tested:
num_layers_list = [2, 3]
epochs_list = [10, 30, 50, 70]
num_units_list = [32, 64, 128, 256, 512]


# Run the automatic script for training the model with different hyperparameter configurations and testing the performance (Q3 score) on the 
# testing set. The resulting hyperparameter set that yields the best performance is printed eventually:
best_accuracy = 0
best_hyperparameters = {}
qian_sejnowski_model = None
for num_layers in num_layers_list:
    for num_units in num_units_list:
        for epochs in epochs_list:
            print(f"Model with {num_layers} layers, {epochs} epochs, and {num_units} units:")
            
            # Train the model with the current hyperparameter configuration:
            model = setup_model(num_units, num_layers, 13 * 21)
            model.fit(X_train, y_train, epochs=epochs, verbose=0)

            predicted = predictions_one_hot_encoding(model.predict(X_test))
            q3 = calculate_q3_score(y_test, predicted)
            
            # Get the accuracy on the testing set and print it:
            print(f"Q3 = {q3}\n")
            
            # If the current hyperparameters are best-performing, save them:
            if q3 > best_accuracy:
                best_accuracy = q3
                best_hyperparameters = {'num_layers': num_layers, 'num_units': num_units, 'epochs': epochs}
                qian_sejnowski_model = model

# Print the best-performing hyperparameter set and its accuracy:
print(f"Best hyperparameters: {best_hyperparameters}")
print(f"Q3 score: {best_accuracy}")

Model with 2 layers, 10 epochs, and 32 units:
10/10 [==============================] - 0s 302us/step
Q3 = 0.6054421768707483

Model with 2 layers, 30 epochs, and 32 units:
10/10 [==============================] - 0s 303us/step
Q3 = 0.5612244897959183

Model with 2 layers, 50 epochs, and 32 units:
10/10 [==============================] - 0s 335us/step
Q3 = 0.5816326530612245

Model with 2 layers, 70 epochs, and 32 units:
10/10 [==============================] - 0s 292us/step
Q3 = 0.5952380952380952

Model with 2 layers, 10 epochs, and 64 units:
10/10 [==============================] - 0s 293us/step
Q3 = 0.5952380952380952

Model with 2 layers, 30 epochs, and 64 units:
10/10 [==============================] - 0s 297us/step
Q3 = 0.5748299319727891

Model with 2 layers, 50 epochs, and 64 units:
10/10 [==============================] - 0s 299us/step
Q3 = 0.5816326530612245

Model with 2 layers, 70 epochs, and 64 units:
10/10 [==============================] - 0s 297us/step
Q3 = 0.5476190476

**We can see that the most optimal performance is when the model contains 2 layers with 32 units in the input layer, and runs for 10 epochs.**

## 1.6. Model Evaluation

In [105]:
# Predict secondary structures using the vbest-performing hyperparameter set:
predictions = predictions_one_hot_encoding(qian_sejnowski_model.predict(X_test))

# Calculate the Q3 score and correlation coefficients of the final model:
q3_score = calculate_q3_score(y_test, predictions)
corr = calculate_correlation_coefficients(y_test, predictions)
print("\nRe-implementation of the model by Qian and Sejnowski:")
print("Q3 Score:", q3_score)
print("Correlation coefficients:")
print(corr)

10/10 [==============================] - 0s 654us/step

Re-implementation of the model by Qian and Sejnowski:
Q3 Score: 0.6054421768707483
Correlation coefficients:
{'alpha-helix': 0.2390353193190812, 'beta-sheet': 0.1521105638433759, 'coil': 0.19618836659118954}


# Task 2: Implement a single improvement to the model

## 2.1. Drop-Out and Learning Rate added (Improvement №1)

In [68]:
# Updated function for initializing the Neural Network model and its layers, that includes drop out and learning rate: 
def setup_improved_model(num_units, input_shape, dropout_rate, learning_rate):
    # Initialize the model (including the drop-out layer):
    model = Sequential([
        Dense(num_units, activation='relu', input_shape=(input_shape,)),
        Dropout(dropout_rate),
        Dense(3, activation='softmax')
    ])

    # Set the model's learning rate, optimizer, loss function and training performance metrics:
    if learning_rate is None:
         model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    else:
        model.compile(optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=learning_rate),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
    return model

In [125]:
# Define lists with hyperparameter configurations of dropout and learning rates:
learning_rates_list = [None, 1e-2, 1e-3, 1e-4]
dropout_rates_list = [0.3, 0.0, 0.1, 0.2, 0.5]

# Run the automatic script for training the model with different drop-out and learning rates and testing the performance (Q3 score) on the 
# testing set. The resulting hyperparameter set that yields the best performance is printed eventually:
best_q3 = 0
best_drop_learing = {}
improved_model_predictions = None
for dropout_rate in dropout_rates_list:
    for learning_rate in learning_rates_list:
        print(f"Model with {dropout_rate} drop-out rate and learning rate of {learning_rate}:")
        
        # Train the model with the current rates configuration:
        model = setup_improved_model(32, 13 * 21, dropout_rate, learning_rate)
        model.fit(X_train, y_train, epochs=10, verbose=0)

        predicted = predictions_one_hot_encoding(model.predict(X_test))
        q3 = calculate_q3_score(y_test, predicted)
        
        # Get the accuracy on the testing set and print it:
        print(f"Q3 = {q3}\n")
        
        # If the current parameters are best-performing, save them:
        if q3 > best_q3:
            improved_model_predictions = predicted
            best_q3 = q3
            best_drop_learing = {'dropout_rate': dropout_rate, 'learning_rate': learning_rate}

# Print the best-performing hyperparameter set and its accuracy:
print(f"Best Drop-Out and Learning Rates: {best_drop_learing}")
print(f"Q3 score: {best_q3}")

Model with 0.3 drop-out rate and learning rate of None:
10/10 [==============================] - 0s 304us/step
Q3 = 0.6224489795918368

Model with 0.3 drop-out rate and learning rate of 0.01:
10/10 [==============================] - 0s 311us/step
Q3 = 0.5884353741496599

Model with 0.3 drop-out rate and learning rate of 0.001:
10/10 [==============================] - 0s 315us/step
Q3 = 0.5884353741496599

Model with 0.3 drop-out rate and learning rate of 0.0001:
10/10 [==============================] - 0s 311us/step
Q3 = 0.48639455782312924

Model with 0.0 drop-out rate and learning rate of None:
10/10 [==============================] - 0s 328us/step
Q3 = 0.5884353741496599

Model with 0.0 drop-out rate and learning rate of 0.01:
10/10 [==============================] - 0s 318us/step
Q3 = 0.5986394557823129

Model with 0.0 drop-out rate and learning rate of 0.001:
10/10 [==============================] - 0s 311us/step
Q3 = 0.5782312925170068

Model with 0.0 drop-out rate and learning r

**As we can see, the model without introducing learning rate performs better, so learning rate does not improve performance. 
The drop-out rate of 0.3, on the other hand, improved the model's Q3 score from 60.5% to 62.2%.**

In [127]:
# Calculate and print the correlation coefficients of model with drop-out rate of 0.3:
print("Correlation coefficients of the model with 0.3 drop-out rate:")
calculate_correlation_coefficients(y_test, improved_model_predictions)

Correlation coefficients of the model with 0.3 drop-out rate:


{'alpha-helix': 0.22938583985479946,
 'beta-sheet': 0.21889586523635784,
 'coil': 0.22793520834446804}

**We can see that the correlation coefficients also improved.**

## 2.2. Cascaded network (Improvement №2)

In [154]:
# Function for applying the sliding window approach on the outputs of the first model:
def cascaded_sliding_window(predictions):
    # Initialize the transformed feature set:
    encoded = np.zeros((len(predictions) - 12, 13 * 3))

    # Apply the sliding window method to store 13 units as one feature:
    for i in range(len(predictions) - 12):
        window = predictions[i:i + 13].flatten()
        encoded[i, :] = window
        
    return encoded

In [174]:
# First, train the model in the same way as in Qian and Sejnowski's work, implemented in Task 1:
first_model = setup_improved_model(32, 13 * 21, 0.3, None)
first_model.fit(X_train, y_train, epochs=10, verbose=0)

# Make predictions on the test set using the first model:
first_model_predictions = predictions_one_hot_encoding(first_model.predict(X_train))
first_model_test = predictions_one_hot_encoding(first_model.predict(X_test))

# Encode the predictions using the sliding window approach:
transformed_train_predictions = cascaded_sliding_window(first_model_predictions)
transformed_test_predictions = cascaded_sliding_window(first_model_test)

# Now, train the second (cascaded) model using the output of the first model as the input for this model:
second_model = setup_model(32, 2, 13 * 3)
second_model.fit(transformed_train_predictions, y_train[:len(transformed_train_predictions)], epochs=10, verbose=0)

# Make predictions on the test set with the cascaded model:
cascaded_predictions = predictions_one_hot_encoding(second_model.predict(transformed_test_predictions))

# Print Q3 score and correlation coefficients of the cascaded model:
q3_score_cascaded = calculate_q3_score(y_test[:len(cascaded_predictions)], cascaded_predictions)
print("Cascaded model:")
print("Q3 Score:", q3_score_cascaded)
print("Correlation coefficients:")
calculate_correlation_coefficients(y_test[:len(cascaded_predictions)], cascaded_predictions)

9/9 [==============================] - 0s 284us/step
Cascaded model:
Q3 Score: 0.6276595744680851
Correlation coefficients:


{'alpha-helix': 0.2743740934102708,
 'beta-sheet': 0.1448173271566131,
 'coil': 0.2795427941487676}

**We can see that the cascaded model further improves the Q3 score (to 62.8%) and correlation coefficients, so it improves the model's performance.**

In [175]:
# Function for running the cascaded networks that will be used as part of future improvements:
def predict_using_cascaded_model(X_train, y_train):
    # First, train the model in the same way as in Qian and Sejnowski's work, implemented in Task 1:
    first_model = setup_improved_model(32, 13 * 21, 0.3, None)
    if y_train is None:
        first_model.fit(X_train, epochs=10, verbose=0)
    else:
        first_model.fit(X_train, y_train, epochs=10, verbose=0)
    
    # Make predictions on the test set using the first model:
    first_model_predictions = predictions_one_hot_encoding(first_model.predict(X_train))
    first_model_test = predictions_one_hot_encoding(first_model.predict(X_test))

    # Encode the predictions using the sliding window approach:
    transformed_train_predictions = cascaded_sliding_window(first_model_predictions)
    transformed_test_predictions = cascaded_sliding_window(first_model_test)
    
    # Now, train the second (cascaded) model using the output of the first model as the input for this model:
    second_model = setup_model(32, 2, 13 * 3)
    second_model.fit(transformed_train_predictions, y_train[:len(transformed_train_predictions)], epochs=10)
    
    # Make predictions on the test set with the cascaded model:
    cascaded_predictions = predictions_one_hot_encoding(second_model.predict(transformed_test_predictions))
    
    # Print Q3 score and correlation coefficients of the cascaded model:
    q3_score_cascaded = calculate_q3_score(y_test[:len(cascaded_predictions)], cascaded_predictions)
    corr_cascaded = calculate_correlation_coefficients(y_test[:len(cascaded_predictions)], cascaded_predictions)
    print("Cascaded model")
    print("Q3 Score:", q3_score_cascaded)
    print("Correlation coefficients:")
    print(corr_cascaded)

## 2.3. Profiling (Improvement №3)

In [176]:
# Start logging the profiling:
tf.profiler.experimental.start(logdir='profiler_logs')

# Run the model that was improved using cascaded networks and drop-out rate:
predict_using_cascaded_model(X_train, y_train)

# Stop logging the profiling:
tf.profiler.experimental.stop()

2024-05-11 22:43:24.773997: I tensorflow/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-05-11 22:43:24.774027: I tensorflow/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.


10/10 [==============================] - 0s 287us/step
Epoch 1/10
524/524 [==============================] - 0s 323us/step - loss: 0.9808 - accuracy: 0.5507
Epoch 2/10
524/524 [==============================] - 0s 318us/step - loss: 0.9233 - accuracy: 0.5962
Epoch 3/10
524/524 [==============================] - 0s 319us/step - loss: 0.9117 - accuracy: 0.6043
Epoch 4/10
524/524 [==============================] - 0s 318us/step - loss: 0.9056 - accuracy: 0.6095
Epoch 5/10
524/524 [==============================] - 0s 313us/step - loss: 0.9016 - accuracy: 0.6095
Epoch 6/10
524/524 [==============================] - 0s 314us/step - loss: 0.8990 - accuracy: 0.6119
Epoch 7/10
524/524 [==============================] - 0s 312us/step - loss: 0.8974 - accuracy: 0.6121
Epoch 8/10
524/524 [==============================] - 0s 318us/step - loss: 0.8956 - accuracy: 0.6138
Epoch 9/10
524/524 [==============================] - 0s 319us/step - loss: 0.8949 - accuracy: 0.6144
Epoch 10/10
9/9 [==========

2024-05-11 22:43:28.573987: I tensorflow/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2024-05-11 22:43:29.096783: I tensorflow/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2024-05-11 22:43:29.099257: I tensorflow/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: profiler_logs/plugins/profile/2024_05_11_22_43_29/RenataBook.local.xplane.pb


In [177]:
# Uncomment the line below and run it to launch the TensorBoard profiling analyzer:
# !tensorboard --logdir=profiler_logs

### Tensorboard has shown the speed performance as follows:

![Speed performance](speed.png)

### We can see that the average step time is slow and should be reduced.

### The memory performance pointed out a bottleneck that should also be mitigated:

![Memory performance](memory.png)

**The following code improves the performance issues identified by profiling:**

In [1]:
# Fnction for improving performance of the dataset through profiling techniques:
def improved_performance_data(X, y):
    # Load the train set using TensorFlow:
    dataset = tf.data.Dataset.from_tensor_slices((X, y))

    # Improve the model's performance by caching the data:
    dataset = dataset.cache()

    # Apply the AUTOTUNE function to parallelize processing of data:
    dataset = dataset.map(mapp, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    dataset = dataset.batch(32)

    # Configure the following elements to be prepared while the current elements are being processed:
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return dataset

# A mapping function:
def mapp(input_features, target_labels):
    return input_features, target_labels

### Run the model improved with profiling, cascaded networks, and drop-out:

In [179]:
# Function for running the model improved with profiling, cascaded networks, and dropout:
def run_improved_model(X_train, y_train, X_test, y_test):
    # Prepare the dataset with improved performance:
    train_set_improved = improved_performance_data(X_train, y_train)
    
    # Setup and train the first model:
    first_model = setup_improved_model(32, 13 * 21, 0.3, None)
    first_model.fit(train_set_improved, epochs=10, verbose=0)
    
    # Make predictions with the first model on both train and test set to train second model:
    first_model_train_predictions = predictions_one_hot_encoding(first_model.predict(X_train))
    first_model_test_predictions = predictions_one_hot_encoding(first_model.predict(X_test))
    
    # Transform the predictions using the sliding window approach:
    transformed_train_predictions = cascaded_sliding_window(first_model_train_predictions)
    transformed_test_predictions = cascaded_sliding_window(first_model_test_predictions)
    
    # Setup and train the second model:
    second_model = setup_improved_model(32, 13 * 3, 0.3, None)  # Adjust the input shape to match sliding window output
    second_model.fit(transformed_train_predictions, y_train[:len(transformed_train_predictions)], epochs=10, verbose=0)
    
    # Make predictions on the test set with the cascaded model:
    cascaded_test_predictions = second_model.predict(transformed_test_predictions)
    cascaded_predictions_encoded = predictions_one_hot_encoding(cascaded_test_predictions)
    
    # Calculate Q3 score and correlation coefficients for the cascaded model:
    q3_score_cascaded = calculate_q3_score(y_test[:len(cascaded_predictions_encoded)], cascaded_predictions_encoded)
    corr_cascaded = calculate_correlation_coefficients(y_test[:len(cascaded_predictions_encoded)], cascaded_predictions_encoded)
    print("Cascaded model:")
    print("Q3 Score:", q3_score_cascaded)
    print("Correlation coefficients:", corr_cascaded)

# Call the function for running the model improved with profiling, cascaded networks, and dropout:
run_improved_model(X_train, y_train, X_test, y_test)

2024-05-11 22:45:04.792543: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [16774,3]
	 [[{{node Placeholder/_1}}]]
2024-05-11 22:45:04.792702: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [16774,3]
	 [[{{node Placeholder/_1}}]]


9/9 [==============================] - 0s 317us/step
Cascaded model:
Q3 Score: 0.6312056737588653
Correlation coefficients: {'alpha-helix': 0.28480123095917265, 'beta-sheet': 0.1625535507407927, 'coil': 0.2785191491098936}


**The model with improved profiling has also improved the speed and memory performance of the model. Furthermore, it increased the Q3 score to 63.1%, and improved correlation coefficients.**

# Task 3: Does the model get similar accuracy on unseen datasets?

**For implementing this task, the RS126 dataset was installed, which contains amino acids and secondary structures of 126 proteins.**

## 3.1. Read and Pre-Process the RS126 dataset

In [180]:
# Function for reading the RS126 dataset from file and storing it as 2 strings - one for amino acids and one for secondary structures, 
# with groups split by spacer '|':
def read_RS126_dataset():
    amino_acids = ""
    secondary_structures = ""
    num_groups = 0  # Counter for the number of groups in the dataset

    # Read the file line by line:
    with open('RS126.data.txt', 'r') as f:
        while True:
            protein_sequence = f.readline().strip()
            structure_sequence = f.readline().strip()

            # Stop looping when the end of file is reached:
            if not protein_sequence or not structure_sequence:
                break

            # If it is not the first protein, separate the current protein's sequences from the previous protein using spacers:
            if amino_acids:
                amino_acids += '|'
                secondary_structures += '|'

            # Re-format the secondary structures to store them in the same way as in the dataset used by Qian and Sejnowski:
            structure_sequence = structure_sequence.replace("H", "h")
            structure_sequence = structure_sequence.replace("E", "e")
            structure_sequence = structure_sequence.replace("C", "_")

            # Store the current protein's amino acids and secondary structures:
            amino_acids += protein_sequence
            secondary_structures += structure_sequence
            num_groups += 1

    return amino_acids, secondary_structures, num_groups

# Store the RS126 dataset as 2 strings - one for amino acids and one for secondary structures, with groups split by spacer '|':
RS126_amino_acids, RS126_secondary_structures, num_RS126_groups = read_RS126_dataset()
print("Number of protein groups in the RS126 dataset:", num_RS126_groups)

Number of protein groups in the RS126 dataset: 126


In [181]:
print(RS126_amino_acids)

APAFSVSPASGASDGQSVSVSVAAAGETYYIAQCAPVGGQDACNPATATSFTTDASGAASFSFTVRKSYAGQTPSGTPVGSVDCATDACNLGAGNSGLNLGHVALTFG|CSVDIQGNDQMQFNTNAITVDKSCKQFTVNLSHPGNLPKNVMGHNWVLSTAADMQGVVTDGMASGLDKDYLKPDDSRVIAHTKLIGSGEKDSVTFDVSKLKEGEQYMFFCTFPGHSALMKGTLTLK|NVYHDGACPEVKPVDNFDWSNYHGKWWEVAKYPNSVEKYGKCGWAEYTPEGKSVKVSNYHVIHGKEYFIEGTAYPVGDSKIGKIYHKLTYGGVTKENVFNVLSTDNKNYIIGYYCKYDEDKKGHQDFVWVLSRSKVLTGEAKTAVENYLIGSPVVDSQKLVYSDFSEAACKVN|AAPCFCSGKPGRGDLWILRGTCPGGYGYTSNCYKWPNICCYPH|SISQQTVWNQMATVRTPLNFDSSKQSFCQFSVDLLGGGISVDKTGDWITLVQNSPISNLLRVAAWKKGCLMVKVVMSGNAAVKRSDWASLVQVFLTNSNSTEHFDACRWTKSEPHSWELIFPIEVCGPNNGFEMWSSEWANQTSWHLSFLVDNPKQSTTFDVLLGISQNFEIAGNTLMPAFSVPQ|METNLFKLSLDDVETPKGSMLDLKISQSKIALPKNTVGGTILRSDLLANFLTEGNFRASVDLQRTHRIKGMIKMVATVGIPENTGIALACAMNSSIRGRASSDIYTICSQDCELWNPACTKAMTMSFNPNPCSDAWSLEFLKRTGFHCDIICVTGWTATPMQDVQVTIDWFISSQECVPRTYCVLNPQNPFVLNRWMGKLTFPQGTSRSVKRMPLSIGGGAGAKSAILMNMPNAVLSMWRYFVGDLVFEVSKMTSPYIKCTVSFFIAFGNLADDTINFEAFPHKLVQFGEIQEKVVLKFSQEEFLTAWSTQVRPATTLLADGCPYLYAMVHDSSVSTIPGDFVIGVKLTIIENMCAYGLN

In [182]:
print(RS126_secondary_structures)

__eeeee_________eeeeeee____eeeeeee_ee__ee________eee_______eeeee___eeeee_____eeeeee______eeeee______________|__eeee___________ee______eeeeeee___________e__eeee____hhhhhhhhhh_hhhh__________e____e_____eeeeee_________eeee__________eeeee__|_eeee_____________hhh__eeeeeeee_________eeeeeeeee____eeeeeeeee__eeeeeeeeeee_______eeeeeeee__eeeeeeeeeeee____eeeeeeeeee____eeeeeeeeee______hhhhhhhhhhhhh_____hhh_ee____hhhh___|_____________eee______________eeee__eeee___|________eeeeeee___________eeeeeee____eee_______eeee___hhhhhhhhe__eeeeeeeeeeee______hhh____eeeeee__________eeeee____eeeeeeeeee_______e____________eeeeeee______eeeeeeee____ee__eee___ee___|_______________________eeeeeeeee_______eeeeeeehhhhh_______hhhhh____e___eeeeee_________eeeeeee_ee______________eeeee_______eeeee________e_hhhhh____eeeeeeee_________eeeeeeeee_________ee_______eeeeeeeeeeee__e_____eeee______eee__eee__hhhh_hhh__eeeeeeeeeeee______e__eeeeee________hhhhh__eeee_______eeeeee_hhh____ee_______________eeeeee___ee______eeeeeeeeeeeeee_____

In [183]:
# Print the types of amino acids and secondary structures found in the RS126 dataset:
print("RS126 amino acid symbols:", set(RS126_amino_acids))
print("RS126 secndary structure symbols:", set(RS126_secondary_structures))

RS126 amino acid symbols: {'H', 'F', 'G', 'Y', 'Q', 'K', 'W', 'V', '|', 'C', 'A', 'R', 'M', 'N', 'I', 'P', 'E', 'L', 'T', 'S', 'D'}
RS126 secndary structure symbols: {'h', '|', '_', 'e'}


In [184]:
# Verify that the amino acid and secondary structure symbols in RS126 and Qian and Sejnowski's datasets are the same:

print("Are the amino acid symbols found in RS126 the same as in Qian and Sejnowski's dataset?")
if set(RS126_amino_acids) == set(train_amino_acids + test_amino_acids):
    print("Yes")
else:
    print("No")

print("\nAre the secondary structure symbols found in RS126 the same as in Qian and Sejnowski's dataset?")
if set(RS126_secondary_structures) == set(train_secondary_structures + test_secondary_structures):
    print("Yes")
else:
    print("No")

Are the amino acid symbols found in RS126 the same as in Qian and Sejnowski's dataset?
Yes

Are the secondary structure symbols found in RS126 the same as in Qian and Sejnowski's dataset?
Yes


In [185]:
# Get the input features and target values for the RS126 dataset using the sliding window approach:
X_RS126, y_RS126 = sliding_window(train_amino_acids, train_secondary_structures)

## 3.2. Make predictions using Qian and Sejnowski's re-implemented model and evaluate the model on the RS126 dataset

In [190]:
# Predict secondary structures:
predictions_RS126 = predictions_one_hot_encoding(qian_sejnowski_model.predict(X_RS126))

# Calculate the Q3 score of the final model:
q3_score = calculate_q3_score(y_RS126, predictions_RS126)
print("RS126 Q3 Score:", q3_score)

# Calculate the correlation coefficients for each secondary structure type and print them:
corr = calculate_correlation_coefficients(y_RS126, predictions_RS126)
print("RS126 Correlation coefficients:")
print(corr)

525/525 [==============================] - 0s 261us/step
RS126 Q3 Score: 0.621139859306069
RS126 Correlation coefficients:
{'alpha-helix': 0.35717298418032395, 'beta-sheet': 0.2499389111541829, 'coil': 0.33924031491107187}


**We can see that the original model, re-implemented as in the work by Qian and Sejnowski, performs even better on the new, unseen RS126 dataset. Its Q3 score on the original testing set used to be 60.5%, but on the RS126 dataset it gave 62.1%. The correlation coefficients are significantly better than on the original testing set also.**

## 3.3. Make predictions using the improved Qian and Sejnowski's model and evaluate the improved model on the RS126 dataset

In [224]:
# Make predictions using the model that was also improved with dropout rates, cascaded networks and profiling on the RS126 dataset:
run_improved_model(X_train, y_train, X_RS126, y_RS126)

2024-05-11 22:53:27.662036: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [16774,273]
	 [[{{node Placeholder/_0}}]]
2024-05-11 22:53:27.662211: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [16774,3]
	 [[{{node Placeholder/_1}}]]


524/524 [==============================] - 0s 191us/step
Cascaded model:
Q3 Score: 0.6183033050948574
Correlation coefficients: {'alpha-helix': 0.3498445867735143, 'beta-sheet': 0.26430250773279196, 'coil': 0.3162224251338081}


**The model improved with profiling, cascaded networks, and dropout, performs slightly lower on the RS126 dataset (Q3 score = 61.8%) compared to the given testing data (Q3 score = 63.1%). However, the scores are very close, indicating similar performance on unseen datasets. Conversely, the correlation coefficients are significantly better than on the original testing set.**

# Task 4: Extend your work to other methods

## 4.1. Support Vector Machines (SVMs)

In [225]:
# List that will be used to convert integer predicted classes into the local encoding format:
local_encoding_mappings = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]

**SVMs with 3 different kernel types were tested to identify the best-performing one:**

In [226]:
# Set up the SVM model with RBF kernel, including scaling of data:
scaler = StandardScaler()
svm_rbf_model = make_pipeline(StandardScaler(), SVC(C=1, kernel='rbf', gamma='scale'))
svm_rbf_model.fit(X_train, np.argmax(y_train, axis=1)) # Convert the encoded labels into integers, as needed to train the SVM model

# Make predictions with the SVM model:
svm_rbf_pred = svm_rbf_model.predict(X_test)

# Convert the integer predictions back into the local encoding format:
svm_rbf_predictions = [local_encoding_mappings[i] for i in svm_rbf_pred]

# Calculate and print the Q3 score and correlation coefficients of the SVM model with RBF kernel:
q3_score = calculate_q3_score(y_test, np.array(svm_rbf_predictions))
corr = calculate_correlation_coefficients(y_test, np.array(svm_rbf_predictions))
print("RBF kernel:")
print("Q3 Score:", q3_score)
print("Correlation coefficients:")
print(corr)

RBF kernel:
Q3 Score: 0.5952380952380952
Correlation coefficients:
{'alpha-helix': 0.2571055793611332, 'beta-sheet': 0.21273644911977285, 'coil': 0.17440231976382167}


In [227]:
# Set up the SVM model with linear kernel, including scaling of data:
scaler = StandardScaler()
svm_linear_model = make_pipeline(StandardScaler(), SVC(C=1, kernel='linear'))
svm_linear_model.fit(X_train, np.argmax(y_train, axis=1)) # Convert the encoded labels into integers, as needed to train the SVM model

# Make predictions with the SVM model:
svm_linear_pred = svm_linear_model.predict(X_test)

# Convert the integer predictions back into the local encoding format:
svm_linear_predictions = [local_encoding_mappings[i] for i in svm_linear_pred]

# Calculate and print the Q3 score and correlation coefficients of the SVM model with linear kernel:
q3_score = calculate_q3_score(y_test, np.array(svm_linear_predictions))
corr = calculate_correlation_coefficients(y_test, np.array(svm_linear_predictions))
print("Linear kernel:")
print("Q3 Score:", q3_score)
print("Correlation coefficients:")
print(corr)

Linear kernel:
Q3 Score: 0.5680272108843537
Correlation coefficients:
{'alpha-helix': 0.2028871879491173, 'beta-sheet': 0.22733087696666757, 'coil': 0.14763922716124703}


In [232]:
# Set up the SVM model with polynomial kernel, including scaling of data:
scaler = StandardScaler()
svm_poly_model = make_pipeline(StandardScaler(), SVC(C=1, kernel='poly', degree=3))
svm_poly_model.fit(X_train, np.argmax(y_train, axis=1)) # Convert the encoded labels into integers, as needed to train the SVM model

# Make predictions with the SVM model:
svm_poly_pred = svm_poly_model.predict(X_test)

# Convert the integer predictions back into the local encoding format:
svm_poly_predictions = [local_encoding_mappings[i] for i in svm_poly_pred]

# Calculate and print the Q3 score and correlation coefficients of the SVM model with polynomial kernel:
q3_score = calculate_q3_score(y_test, np.array(svm_poly_predictions))
corr = calculate_correlation_coefficients(y_test, np.array(svm_poly_predictions))
print("Polynomial kernel:")
print("Q3 Score:", q3_score)
print("Correlation coefficients:")
print(corr)

Polynomial kernel:
Q3 Score: 0.6156462585034014
Correlation coefficients:
{'alpha-helix': 0.21913580616316516, 'beta-sheet': 0.2564149493684171, 'coil': 0.18721724535825374}


**We can see that the SVM model gives best results with the polynomial kernel, resulting in the Q3 score of 61.6%.**

In [234]:
print("Performance of the SVM model with polynomial kernel:")

# Print the Q3 score and correlation coefficients of the SVM model with polynomial kernel:
q3_score = calculate_q3_score(y_test, np.array(svm_poly_predictions))
corr = calculate_correlation_coefficients(y_test, np.array(svm_poly_predictions))
print("\nOriginally given testing set:")
print("Q3 Score:", q3_score)
print("Correlation coefficients:")
print(corr)

# Additionally, evaluate the model with polynomial kernel on the RS126 dataset also:
svm_poly_predictions_rs126 = [local_encoding_mappings[i] for i in svm_poly_model.predict(X_RS126)]
q3_score = calculate_q3_score(y_RS126, np.array(svm_poly_predictions_rs126))
corr = calculate_correlation_coefficients(y_RS126, np.array(svm_poly_predictions_rs126))
print("\nRS126 dataset:")
print("Q3 Score:", q3_score)
print("Correlation coefficients:")
print(corr)

Performance of the SVM model with polynomial kernel:

Originally given testing set:
Q3 Score: 0.6156462585034014
Correlation coefficients:
{'alpha-helix': 0.21913580616316516, 'beta-sheet': 0.2564149493684171, 'coil': 0.18721724535825374}

RS126 dataset:
Q3 Score: 0.9555264099201145
Correlation coefficients:
{'alpha-helix': 0.9516224498471313, 'beta-sheet': 0.9191009824329259, 'coil': 0.9139210259804006}


**The results indicate that on the given testing set (from Brookhaven National Laboratory), the performance of the SVM model is similar to that of the re-implemented neural network. However, on the RS126 dataset, the SVM model significantly outperforms the neural network model, resulting in the Q3 score of 95.6%, which is an exceptionally high performance. The correlation coefficients are also much higher than of the neural network re-implementation. This leads to the conclusion that the SVM model outperforms the re-implementation of Qian and Sejnowski's network.**

## 4.2. Convolutional Neural Networks (CNNs)

In [245]:
# Function for setting up a CNN model:
def setup_cnn_model(dropout_rate):

    # Initialize the CNN model including the dropout layer:
    model = Sequential([
        Conv1D(64, 3, activation='relu', padding='same', input_shape=(13, 21)),
        BatchNormalization(),
        Conv1D(128, 3, activation='relu', padding='same'),
        MaxPooling1D(2),
        Dropout(dropout_rate),
        Conv1D(256, 3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(2),
        Flatten(),
        Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        Dropout(0.5),
        Dense(3, activation='softmax')
    ])

    # Set the model's optimizer and loss function, along with the training metric:
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Re-shape the data for further model training:
X_train_reshaped = X_train.reshape((-1, 13, 21))
X_test_reshaped = X_test.reshape((-1, 13, 21))

In [246]:
# Define the lists with hyperparameter configurations for the CNN model:
cnn_dropout_rates = [0, 0.1, 0.3, 0.5, 0.7]
num_epochs_list = [5, 10, 15, 20, 30, 50]

# Run the automatic script for identifying the most optimal hyperparameter set of the CNN model:
best_num_epochs = 0
best_score = 0
best_dropout_rate = None
best_cnn_predictions = None
best_cnn_model = None
for num_epochs in num_epochs_list:
    for dropout_rate in cnn_dropout_rates:
        
        # Set-up and train the CNN model:
        cnn_model = setup_cnn_model(dropout_rate)
        cnn_model.fit(X_train_reshaped, y_train, epochs=num_epochs, batch_size=32, verbose=0, validation_split=0.2)
        
        # Predict secondary structures using the CNN model:
        cnn_predictions = predictions_one_hot_encoding(cnn_model.predict(X_test_reshaped))
    
        # Calculate and print the Q3 score of the CNN model:
        q3_score = calculate_q3_score(y_test, np.array(cnn_predictions))
    
        if q3_score > best_score:
            best_num_epochs = num_epochs
            best_dropout_rate = dropout_rate
            best_score = q3_score
            best_cnn_predictions = cnn_predictions
            best_cnn_model = cnn_model
            
        print(f"Q3 Score of CNN model with {num_epochs} epochs and {dropout_rate} dropout rate: {q3_score}")
    

# Print the best-perorming hyperparameter set and its Q3 score:
print(f"\nBest for the CNN model: {best_num_epochs} epochs and {best_dropout_rate} dropout rate")
print("Q3 Score:", best_score)

10/10 [==============================] - 0s 892us/step
Q3 Score of CNN model with 5 epochs and 0 dropout rate: 0.5306122448979592
10/10 [==============================] - 0s 882us/step
Q3 Score of CNN model with 5 epochs and 0.1 dropout rate: 0.6190476190476191
10/10 [==============================] - 0s 802us/step
Q3 Score of CNN model with 5 epochs and 0.3 dropout rate: 0.6122448979591837
10/10 [==============================] - 0s 807us/step
Q3 Score of CNN model with 5 epochs and 0.5 dropout rate: 0.5714285714285714
10/10 [==============================] - 0s 874us/step
Q3 Score of CNN model with 5 epochs and 0.7 dropout rate: 0.6054421768707483
10/10 [==============================] - 0s 885us/step
Q3 Score of CNN model with 10 epochs and 0 dropout rate: 0.5680272108843537
10/10 [==============================] - 0s 876us/step
Q3 Score of CNN model with 10 epochs and 0.1 dropout rate: 0.6360544217687075
10/10 [==============================] - 0s 785us/step
Q3 Score of CNN model w

In [249]:
print("Performance of the CNN model:")

# Print the Q3 score and correlation coefficients of the SVM model with polynomial kernel:
q3_score = calculate_q3_score(y_test, np.array(best_cnn_predictions))
corr = calculate_correlation_coefficients(y_test, np.array(best_cnn_predictions))
print("\nOriginally given testing set:")
print("Q3 Score:", q3_score)
print("Correlation coefficients:")
print(corr)

# Additionally, evaluate the model with polynomial kernel on the RS126 dataset also:
X_RS126_reshaped = X_RS126.reshape((-1, 13, 21))
cnn_predictions_rs126 = predictions_one_hot_encoding(best_cnn_model.predict(X_RS126_reshaped))
q3_score = calculate_q3_score(y_RS126, np.array(cnn_predictions_rs126))
corr = calculate_correlation_coefficients(y_RS126, np.array(cnn_predictions_rs126))
print("\nRS126 dataset:")
print("Q3 Score:", q3_score)
print("Correlation coefficients:")
print(corr)

Performance of the CNN model:

Originally given testing set:
Q3 Score: 0.6530612244897959
Correlation coefficients:
{'alpha-helix': 0.34759792205288836, 'beta-sheet': 0.1986962510032892, 'coil': 0.3723908918406072}
525/525 [==============================] - 0s 768us/step

RS126 dataset:
Q3 Score: 0.8856563729581495
Correlation coefficients:
{'alpha-helix': 0.8142531573434764, 'beta-sheet': 0.8191136390352364, 'coil': 0.799890847414605}


**The CNN model outperformed all other models when predicting on the originally given testing set (from Brookhaven National Laboratory), with the resulting Q3 score of 65.3%. However, with the RS126 dataset, it results in second highest performance, as the highest Q3 score on the RS126 dataset was achieved by the SVM model.**